In [8]:
# pip install mysqlclient
# MySQL사용을 위해 위의 install 작업이 필요함
import MySQLdb
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
# 주소저장을 위한 객체 생성
conn = MySQLdb.connect(**config)
print(conn)
# 작업이 끝난 후엔 반드시 close (계속 켜놓으면 서버가 메모리잡아먹음!)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000000045BF648>


In [ ]:
# import MySQLdb
# config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
# conn = MySQLdb.connect(**config)
# print(conn)
# conn.close()

In [9]:
# select
conn = MySQLdb.connect(**config)
cursor = conn.cursor()
#print(cursor)

# 파이썬, MySQL에 명령어전달 작업
sql = "select empno, ename, sal, job, deptno from scott_emp"
# sql 객체의 값을 가져와서 실행시킨 후 cursor로 옮기기 지시?? = execute
cursor.execute(sql)

# for data in cursor:
#      print(data)
#     print("%d,%s,%d,%s,%d"%data)
    
for (empno, ename, sal, job, deptno) in cursor:
    print(empno, ename, sal, job, deptno)
conn.close()

1 john 2500.0 SALESMAN None
2 tom 700.0 clerk 20
3 jane 1000.0 None None
4 Jhon 2000.0 None None
7369 SMITH 800.0 CLERK 20
7499 ALLEN 1600.0 SALESMAN 30
7521 WARD 1250.0 SALESMAN 30
7566 JONES 2975.0 MANAGER 20
7654 MARTIN 1250.0 SALESMAN 30
7698 BLAKE 2850.0 MANAGER 30
7782 CLARK 2450.0 MANAGER 10
7788 SCOTT 3000.0 ANALYST 20
7839 KING 5000.0 PRESIDENT 10
7844 TURNER 1500.0 SALESMAN 30
7876 ADAMS 1100.0 CLERK 20
7900 JAMES 950.0 CLERK 30
7902 FORD 3000.0 ANALYST 20
7934 MILLER 1300.0 CLERK 10


In [10]:
# insert 1
conn = MySQLdb.connect(**config)
cursor = conn.cursor()
sql = """insert into scott_emp(empno, ename, sal, job, mgr, deptno, hiredate)
        values(2, "tom", 700, "clerk", 7788, 20, now())"""
cursor.execute(sql)
# commit 까지해야 insert 완료
conn.commit()
conn.close()
# Tom의 정보가 생긴게 MySQL로는 확인이 됌.

IntegrityError: (1062, "Duplicate entry '2' for key 'PRIMARY'")

In [11]:
# insert 2
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

v_empno = 3
v_ename = "jane"
v_sal = 1000

sql = "insert into scott_emp(empno, ename, sal) values(" + str(v_empno) \
     + ",'" + v_ename + "'," + str(v_sal) + ")"
print(sql)
cursor.execute(sql)
# commit 까지해야 insert 완료
conn.commit()
conn.close()

insert into scott_emp(empno, ename, sal) values(3,'jane',1000)


IntegrityError: (1062, "Duplicate entry '3' for key 'PRIMARY'")

In [12]:
# insert 3
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

v_empno = 4
v_ename = "Jhon"
v_sal = 2000

sql = "insert into scott_emp(empno, ename, sal) values(%s, %s, %s)"  # <- 요런방법을 애용할 것! (튜플로 묶어서 변수값은 따로 전달하는 방법)
sql_data = (str(v_empno), str(v_ename), str(v_sal))

cursor.execute(sql, sql_data)
# commit 까지해야 insert 완료
conn.commit()
conn.close()

IntegrityError: (1062, "Duplicate entry '4' for key 'PRIMARY'")

In [2]:
# delete
import MySQLdb
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

sql = "delete from scott_emp where empno between 2 and 4"
cursor.execute(sql)
conn.commit()
conn.close()

## csv파일과 MySQL 활용

    CREATE TABLE IF NOT EXISTS suppliers(
        supplier_name    varchar(20),
        invoice_number   varchar(20),
        part_number      varchar(20),
        cost             float,
        purchase_date    date
    );

In [6]:
import csv
from datetime import datetime, date

In [19]:
import MySQLdb
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

sql = "delete from suppliers"
cursor.execute(sql)
conn.commit()

In [21]:
import MySQLdb
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
# 주소저장을 위한 객체 생성
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

import csv
from datetime import datetime, date

file = csv.reader(open("data/input.csv"))
# print(file)

# 현재 위치의 data를 가져와서 첫번째 줄 읽기: next ???? 그다음에 next하면 두번째줄.. 이런 식으로 이어짐
# print(next(file))
# print(next(file))
# print(next(file))

header = next(file)
for row in file:   # 반복문 자체의 의미때문에 따로 next안써도 됌
    data = []
# 날짜만 내 입맛에 맞게 형식을 바꾸겠다
    for col in range(len(header)):   # 열의 개수? 길이만큼 반복하겠다는 뜻
        if col == 4:
            tmp = datetime.strptime(row[col], "%Y-%m-%d")
            data.append(tmp)
        else:
            data.append(row[col])
#     print(data)
    cursor.execute("insert into suppliers values(%s,%s,%s,%s, %s)", data)
    
conn.commit()

In [15]:
cursor.execute("select * from suppliers")
for row in cursor:
    output =[]
    for idx in range(len(row)):
        output.append(str(row[idx]))
    print(output)

In [18]:
import MySQLdb
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"dbexam"}
# 주소저장을 위한 객체 생성
conn = MySQLdb.connect(**config)
cursor = conn.cursor()

# mysql_output.csv 파일생성
filewriter = csv.writer(open("data/mysql_output.csv", "w"), delimiter=",")

header = ["Supplier Name", "Invoice Number", "Part Number", "Cost", "Purchase Date"]
filewriter.writerow(header)

sql = "select * from suppliers where cost > 500000"
cursor.execute(sql)
for row in cursor:
    filewriter.writerow(row)

conn.close()